In [ ]:
from transformers import BlipProcessor, BlipForQuestionAnswering, BartTokenizer, BartForConditionalGeneration
from PIL import Image
import torch
import pandas as pd
from tqdm import tqdm
from sklearn.model_selection import train_test_split
import csv
from pathlib import Path

# Output CSV
output_path = Path("vqa_test_predictions.csv")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load BLIP
processor = BlipProcessor.from_pretrained("Salesforce/blip-vqa-base")
model = BlipForQuestionAnswering.from_pretrained("Salesforce/blip-vqa-base").to(device)

train_df = pd.read_csv("/kaggle/input/vrmini2/train_split.csv")
test_df = pd.read_csv("/kaggle/input/vrmini2/test_split.csv")

# Evaluate on test set
predictions = []
references = test_df['answer'].tolist()

for idx, row in tqdm(test_df.iterrows(), total=len(test_df), desc="Running VQA on Test Set"):
    try:
        image_path = row['full_path']
        question = row['question']
        ground_truth = row['answer']

        image = Image.open(image_path).convert("RGB")
        inputs = processor(image, question, return_tensors="pt").to(device)
        output = model.generate(**inputs)
        answer = processor.decode(output[0], skip_special_tokens=True)
    except Exception as e:
        print(f"Failed on {row['full_path']}: {e}")
        answer = ""
    predictions.append(answer)


test_df['predicted_answer'] = predictions
test_df['correct'] = test_df['predicted_answer'].str.strip().str.lower() == test_df['answer'].str.strip().str.lower()

# Save to CSV
test_df.to_csv(output_path, index=False)
print(f"Predictions saved to {output_path}")


2025-05-15 10:16:20.988014: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747304181.221322      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747304181.290554      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


preprocessor_config.json:   0%|          | 0.00/445 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/592 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/4.56k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.54G [00:00<?, ?B/s]

Running VQA on Test Set: 100%|██████████| 2410/2410 [03:54<00:00, 10.28it/s]

Predictions saved to vqa_test_predictions.csv


In [ ]:
!pip install bert-score rouge-score nltk
import pandas as pd
from sklearn.metrics import f1_score
from bert_score import score as bert_score
from rouge_score import rouge_scorer
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from tqdm import tqdm
tqdm.pandas()

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.4 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.3 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 30.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.3 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 80.1 MB/s eta 0:00:00:00:0100:01
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=d325a0b2792bf657ff0afd36b04afdfee052a05720be574e2bbaca12aedddcf4
  Stored in directory: /root/.cache/pip/wheels/1e/19/43/8a442dc83660ca25e163e1bd1

In [ ]:
# Load CSV
df = pd.read_csv("/kaggle/working/vqa_test_predictions.csv")  # Replace with your CSV file path

# Normalize
def normalize_text(s):
    return str(s).strip().lower()

df['answer'] = df['answer'].astype(str).apply(normalize_text)
df['predicted_answer'] = df['predicted_answer'].astype(str).apply(normalize_text)

# Token-level macro F1
def compute_token_f1(pred, gt):
    pred_tokens = pred.split()
    gt_tokens = gt.split()
    common = set(pred_tokens) & set(gt_tokens)
    if not pred_tokens or not gt_tokens:
        return 0.0
    precision = len(common) / len(pred_tokens)
    recall = len(common) / len(gt_tokens)
    if precision + recall == 0:
        return 0.0
    return 2 * precision * recall / (precision + recall)
df['token_f1'] = df.progress_apply(lambda row: compute_token_f1(row['predicted_answer'], row['answer']), axis=1)
avg_token_f1 = df['token_f1'].mean()

# Accuracy
accuracy = (df['answer'] == df['predicted_answer']).mean()

# ROUGE Score
rouge = rouge_scorer.RougeScorer(['rougeL'], use_stemmer=True)
df['rougeL'] = df.progress_apply(lambda row: rouge.score(row['answer'], row['predicted_answer'])['rougeL'].fmeasure, axis=1)
avg_rougeL = df['rougeL'].mean()

# BERTScore (Precision/Recall/F1)
P, R, F1 = bert_score(df['predicted_answer'].tolist(), df['answer'].tolist(), lang='en', verbose=True)
bert_f1 = F1.mean().item()

# BLEU Score
smooth_fn = SmoothingFunction().method1
df['bleu'] = df.progress_apply(lambda row: sentence_bleu([row['answer'].split()], row['predicted_answer'].split(), smoothing_function=smooth_fn), axis=1)
avg_bleu = df['bleu'].mean()

print(f"Token F1      : {avg_token_f1:.4f}")
print(f"Accuracy      : {accuracy:.4f}")
print(f"ROUGE-L       : {avg_rougeL:.4f}")
print(f"BERTScore F1  : {bert_f1:.4f}")
print(f"BLEU          : {avg_bleu:.4f}")



tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

...

computing greedy matching.


  0%|          | 0/38 [00:00<?, ?it/s]

done in 1.82 seconds, 1320.59 sentences/sec


100%|██████████| 2410/2410 [00:00<00:00, 14996.15it/s]

Accuracy      : 0.3415
BERTScore F1  : 0.9449
ROUGE-L       : 0.3560
Token F1      : 0.3502
BLEU          : 0.0626
